## Possible Outcomes of NBA Playoff Series

In this short notebook, we will develop some Python code to generate the possible outcomes of an NBA playoff series. We will use this code in other notebooks to compute playoff series win probabilities.

We will use the `permutations()` function in the [`itertools`](https://docs.python.org/3/library/itertools.html#module-itertools) module from the Python standard library.

In [1]:
import itertools
from collections import Counter

Imagine that we have a best-of-5 playoff series. (Prior to the 2002-3 NBA Season, first round playoff games were best-of-5.) Suppose we represent wins from the perspective of the team which has series home court advantage. We can represent wins and losses by that team as yes/no variables. A `'Y'` represents a win by that team, and a `'N'` represents a loss.

Let's look at one possible sequence of outcomes for a 5-game series.

In [2]:
raw_outcome = ('N', 'N', 'Y', 'N', 'Y')

### Creating Valid Playoff Series Outcomes

In this case, the series should never have gotten to 5 games. The team with series home court advantage would have been eliminated after the fourth game. We want to adjust the above outcome to reflect that the series should have ended after 4 games.

Here's how we can do that. First, we need a function to find the last `'Y'` or `'N'` character. We can use some Python [slicing](https://www.dotnetperls.com/slice-python) with the usual Python [`index()`](https://docs.python.org/3.6/tutorial/datastructures.html) method to find the last occurrence of the item. This method will work on a Python seqeuence implementing the `index()` method (in particular, standard `list` or `tuple` objects).

In [3]:
def last_index(outcome, c):
    """Index of last occurrence of an item in a list or tuple"""
    return len(outcome) - outcome[::-1].index(c)

Suppose we know that the team without series home court advantage won this playff series. Here's how we use our function to find the decisive game.

In [4]:
last_index(raw_outcome, 'N')

4

From this we know that the series should had ended after 4 games. Let's use slicing again to chop off the game which would never have actually been played.

In [5]:
raw_outcome[:last_index(raw_outcome, 'N')]

('N', 'N', 'Y', 'N')

We can turn this idea into a general function. The function will accept a sequence of games (represented by `'Y'` and `'N'` characters representing wins or losses by the team with series home court advantage). It will return the truncated outcome representing the games that would have actually been played.

In [6]:
def valid_series(outcome, *, best_of, hca_wins):
    """Adjust raw outcome to reflect valid playoff series outcome"""
    assert best_of in (5, 7)
    c = 'Y' if hca_wins else 'N'  # Character for counting wins
    wins = 4 if best_of == 7 else 3
    assert outcome.count(c) == wins  # Sanity check the input
    return outcome[:last_index(outcome, c)]

In [7]:
valid_series(raw_outcome, best_of=5, hca_wins=False)

('N', 'N', 'Y', 'N')

Notice that if we happen to pass in a sequence that is already valid, we get back the same (valid) sequence.

In [8]:
valid_series(('N', 'N', 'Y', 'N'), best_of=5, hca_wins=False)

('N', 'N', 'Y', 'N')

### Generating Possible Outcomes

Now it's time to generate all the possible playoff series outcomes.

Continuing with our best-of-5 series as an example, let's generate all possible sequences with 3 wins and 2 losses.

In [9]:
wins = 'Y'*3 + 'N'*2
set(itertools.permutations(wins))

{('N', 'N', 'Y', 'Y', 'Y'),
 ('N', 'Y', 'N', 'Y', 'Y'),
 ('N', 'Y', 'Y', 'N', 'Y'),
 ('N', 'Y', 'Y', 'Y', 'N'),
 ('Y', 'N', 'N', 'Y', 'Y'),
 ('Y', 'N', 'Y', 'N', 'Y'),
 ('Y', 'N', 'Y', 'Y', 'N'),
 ('Y', 'Y', 'N', 'N', 'Y'),
 ('Y', 'Y', 'N', 'Y', 'N'),
 ('Y', 'Y', 'Y', 'N', 'N')}

The `permutations()` function returns an [iterator](https://dbader.org/blog/python-iterators) of all the possible permutations of the 5-character string. Many of the items returned by this iterator would be duplicates. That is why we use `set` above to get the unique outcomes.

We can use our function above to prune each possible permutation and make sure it is a valid NBA playoff outcome. Here's a general function that does what we need.

In [10]:
def playoff_outcomes(*, best_of, hca_wins):
    """Possible playoff series outcomes"""
    if best_of not in (5, 7):
        raise ValueError('playoff series must be best of 5 or 7 games')
    if hca_wins:
        yes = 4 if best_of == 7 else 3
        no = best_of - yes
    else:
        no = 4 if best_of == 7 else 3
        yes = best_of - no
    wins = 'Y'*yes + 'N'*no
    raw_outcomes = set(outcome for outcome in itertools.permutations(wins))
    outcomes = {
        valid_series(outcome, best_of=best_of, hca_wins=hca_wins)
        for outcome in raw_outcomes
    }
    return outcomes

Here are the possible outcomes for a best-of-5 series, where the team without series home court advantage wins.

In [11]:
playoff_outcomes(best_of=5, hca_wins=False)

{('N', 'N', 'N'),
 ('N', 'N', 'Y', 'N'),
 ('N', 'N', 'Y', 'Y', 'N'),
 ('N', 'Y', 'N', 'N'),
 ('N', 'Y', 'N', 'Y', 'N'),
 ('N', 'Y', 'Y', 'N', 'N'),
 ('Y', 'N', 'N', 'N'),
 ('Y', 'N', 'N', 'Y', 'N'),
 ('Y', 'N', 'Y', 'N', 'N'),
 ('Y', 'Y', 'N', 'N', 'N')}

In [12]:
Counter(len(outcome) for outcome in playoff_outcomes(best_of=5, hca_wins=False))

Counter({5: 6, 4: 3, 3: 1})

Here are the possible outcomes of a best-of-7 series, where the team with series home court advantage wins.

In [13]:
playoff_outcomes(best_of=7, hca_wins=True)

{('N', 'N', 'N', 'Y', 'Y', 'Y', 'Y'),
 ('N', 'N', 'Y', 'N', 'Y', 'Y', 'Y'),
 ('N', 'N', 'Y', 'Y', 'N', 'Y', 'Y'),
 ('N', 'N', 'Y', 'Y', 'Y', 'N', 'Y'),
 ('N', 'N', 'Y', 'Y', 'Y', 'Y'),
 ('N', 'Y', 'N', 'N', 'Y', 'Y', 'Y'),
 ('N', 'Y', 'N', 'Y', 'N', 'Y', 'Y'),
 ('N', 'Y', 'N', 'Y', 'Y', 'N', 'Y'),
 ('N', 'Y', 'N', 'Y', 'Y', 'Y'),
 ('N', 'Y', 'Y', 'N', 'N', 'Y', 'Y'),
 ('N', 'Y', 'Y', 'N', 'Y', 'N', 'Y'),
 ('N', 'Y', 'Y', 'N', 'Y', 'Y'),
 ('N', 'Y', 'Y', 'Y', 'N', 'N', 'Y'),
 ('N', 'Y', 'Y', 'Y', 'N', 'Y'),
 ('N', 'Y', 'Y', 'Y', 'Y'),
 ('Y', 'N', 'N', 'N', 'Y', 'Y', 'Y'),
 ('Y', 'N', 'N', 'Y', 'N', 'Y', 'Y'),
 ('Y', 'N', 'N', 'Y', 'Y', 'N', 'Y'),
 ('Y', 'N', 'N', 'Y', 'Y', 'Y'),
 ('Y', 'N', 'Y', 'N', 'N', 'Y', 'Y'),
 ('Y', 'N', 'Y', 'N', 'Y', 'N', 'Y'),
 ('Y', 'N', 'Y', 'N', 'Y', 'Y'),
 ('Y', 'N', 'Y', 'Y', 'N', 'N', 'Y'),
 ('Y', 'N', 'Y', 'Y', 'N', 'Y'),
 ('Y', 'N', 'Y', 'Y', 'Y'),
 ('Y', 'Y', 'N', 'N', 'N', 'Y', 'Y'),
 ('Y', 'Y', 'N', 'N', 'Y', 'N', 'Y'),
 ('Y', 'Y', 'N', 'N', 'Y', 'Y

In [14]:
Counter(len(outcome) for outcome in playoff_outcomes(best_of=7, hca_wins=True))

Counter({7: 20, 6: 10, 5: 4, 4: 1})